# Review Rec Bot

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loguru import logger
import json

import sys
sys.path.insert(0, '..')

In [3]:
import os

from tqdm.notebook import tqdm
import pandas as pd

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

# Arguments

In [6]:
from src.run.args import RunInputArgs

ARGS = RunInputArgs(
    EXPERIMENT_NAME="Review Rec Bot - Yelp Review Rec Bot",
    RUN_NAME="009_hybrid_retriever",
    RUN_DESCRIPTION="""
# Hybrid Retriever (keyword and vector) - Simple Fusion

Implemented via QueryFusionRetriever
Ref: https://docs.llamaindex.ai/en/stable/examples/retrievers/reciprocal_rerank_fusion/
Use Fusion Mode = Simple which just get the max score from retrievers result

## Changelog
Compared to 008:
- Try new retriever
""",
    TESTING=False,
    LOG_TO_MLFLOW=True,
    OBSERVABILITY=True,
    RECREATE_INDEX=True,
    RECREATE_RETRIEVAL_EVAL_DATASET=True,
    RECREATE_RESPONSE_EVAL_DATASET=True,
    DEBUG=False,
)

ARGS

{
  "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
  "RUN_NAME": "009_hybrid_retriever",
  "RUN_DESCRIPTION": "\n# Hybrid Retriever (keyword and vector)\n\nImplemented via QueryFusionRetriever\nRef: https://docs.llamaindex.ai/en/stable/examples/retrievers/reciprocal_rerank_fusion/\n\n## Changelog\nCompared to 008:\n- Try new retriever\n",
  "TESTING": false,
  "DEBUG": false,
  "OBSERVABILITY": true,
  "LOG_TO_MLFLOW": true,
  "RECREATE_INDEX": true,
  "RECREATE_RETRIEVAL_EVAL_DATASET": true,
  "RECREATE_RESPONSE_EVAL_DATASET": true
}

# Load config

In [7]:
from src.run.cfg import RunConfig

In [8]:
cfg = RunConfig()
cfg.init(ARGS)

2024-08-09 10:39:13.995 | INFO     | src.run.cfg:init:125 - Starting Observability server with Phoenix...
INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/dvquys/.phoenix


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


2024-08-09 10:39:19.747 | INFO     | src.run.cfg:init:143 - Setting up MLflow experiment Review Rec Bot - Yelp Review Rec Bot - run 009_hybrid_retriever...
2024-08-09 10:39:19.959 | INFO     | src.run.cfg:init:152 - Notebook-generated artifacts are persisted at data/009_hybrid_retriever
2024-08-09 10:39:19.960 | INFO     | src.run.cfg:init:158 - ARGS.RECREATE_INDEX=True -> Overwriting db_collection and storage_context_persist_dp...


In [9]:
cfg

/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{
  "args": {
    "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
    "RUN_NAME": "009_hybrid_retriever",
    "RUN_DESCRIPTION": "\n# Hybrid Retriever (keyword and vector)\n\nImplemented via QueryFusionRetriever\nRef: https://docs.llamaindex.ai/en/stable/examples/retrievers/reciprocal_rerank_fusion/\n\n## Changelog\nCompared to 008:\n- Try new retriever\n",
    "TESTING": false,
    "DEBUG": false,
    "OBSERVABILITY": true,
    "LOG_TO_MLFLOW": true,
    "RECREATE_INDEX": true,
    "RECREATE_RETRIEVAL_EVAL_DATASET": true,
    "RECREATE_RESPONSE_EVAL_DATASET": true
  },
  "app_name": "review_rec_bot",
  "storage_context_persist_dp": "data/009_hybrid_retriever/storage_context",
  "db_collection": "review_rec_bot__009_hybrid_retriever__huggingface__Snowflake_snowflake_arctic_embed_m_v1_5",
  "notebook_cache_dp": "data/009_hybrid_retriever",
  "data_fp": "../data/yelp_dataset/sample/sample_100_biz/denom_review.parquet",
  "llm_cfg": {
    "llm_provider": "togetherai",
    "llm_

## Set up logger to collect additional info

In [10]:
collect_fp = f"{cfg.notebook_cache_dp}/collect.log"
logger.add(collect_fp, filter=lambda record: "[COLLECT]" in record['message'], mode='w')

1

# Load input data

In [11]:
data = pd.read_parquet(cfg.data_fp)
data = data.assign(
    biz_categories=lambda df: df['biz_categories'].str.split(', '),
    date=lambda df: df['date'].dt.strftime('%Y-%m-%dT%H:%M:%S')
)
logger.info(f"[COLLECT] {len(data)=}")
data.iloc[0]

2024-08-09 10:39:20.140 | INFO     | __main__:<module>:6 - [COLLECT] len(data)=5240


review_id                                      L0jv8c2FbpWSlfNC6bbUEA
user_id                                        bFPdtzu11Oi0f92EAcjqmg
business_id                                    IDtLPgUrqorrpqSLdfMhZQ
review_stars                                                        5
useful                                                              0
funny                                                               0
cool                                                                0
text                What a great addition to the Funk Zone!  Grab ...
date                                              2016-10-13T22:50:47
biz_name                                         Helena Avenue Bakery
biz_address                                     131 Anacapa St, Ste C
biz_city                                                Santa Barbara
biz_state                                                          CA
biz_postal_code                                                 93101
biz_latitude        

In [12]:
input_data = data
if ARGS.TESTING:
    input_data = data[:20]
logger.info(f"[COLLECT] {len(input_data)=}")

2024-08-09 10:39:20.168 | INFO     | __main__:<module>:4 - [COLLECT] len(input_data)=5240


In [13]:
input_data.columns

Index(['review_id', 'user_id', 'business_id', 'review_stars', 'useful',
       'funny', 'cool', 'text', 'date', 'biz_name', 'biz_address', 'biz_city',
       'biz_state', 'biz_postal_code', 'biz_latitude', 'biz_longitude',
       'biz_stars', 'biz_review_count', 'biz_is_open', 'biz_attributes',
       'biz_categories', 'biz_hours'],
      dtype='object')

# Prepare documents

In [14]:
from llama_index.core import Document

documents = []
embedding_visible_metadata = ["review_stars", "biz_name", "biz_address", "biz_city", "biz_state", "biz_categories"]
excluded_embed_metadata_keys = [k for k in input_data.columns if k not in embedding_visible_metadata]

for i, row in tqdm(input_data.iterrows(), total=len(input_data)):
    record = row.to_dict()
    text = record['text']
    metadata = {k: v for k, v in record.items() if k not in ('text')}
    
    doc = Document(
        text=text,
        metadata=metadata,
        excluded_embed_metadata_keys=excluded_embed_metadata_keys,
        excluded_llm_metadata_keys=excluded_embed_metadata_keys
    )
    documents.append(doc)

logger.info(f"[COLLECT] {len(documents)=}")

  0%|          | 0/5240 [00:00<?, ?it/s]

2024-08-09 10:39:20.715 | INFO     | __main__:<module>:20 - [COLLECT] len(documents)=5240


#### Check document embedding text

In [15]:
from llama_index.core.schema import MetadataMode

In [16]:
document = documents[0]
print(document.get_content(metadata_mode=MetadataMode.EMBED))

review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries']

What a great addition to the Funk Zone!  Grab a bite, grab some tastings, life is good. Right next door to the Santa Barbara Wine Collective, in fact it actually shares the same tables.  We had a fabulous savory croissant.


# Set up LLM

In [17]:
llm, embed_model = cfg.setup_llm()

Some weights of BertModel were not initialized from the model checkpoint at Snowflake/snowflake-arctic-embed-m-v1.5 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
print(cfg.llm_cfg.model_dump_json(indent=2))

{
  "llm_provider": "togetherai",
  "llm_model_name": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
  "embedding_provider": "huggingface",
  "embedding_model_name": "Snowflake/snowflake-arctic-embed-m-v1.5",
  "embedding_model_dim": 768,
  "ollama__host": "192.168.100.14",
  "ollama__port": 11434
}


In [19]:
from llama_index.core import Settings
Settings.embed_model = embed_model
Settings.llm = llm

# Vector Store

In [20]:
import qdrant_client
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

from src.run.orchestrator import RunOrchestrator

In [21]:
qdrantdb = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)
aqdrantdb = qdrant_client.AsyncQdrantClient(
    host="localhost",
    port=6333
)

RunOrchestrator.setup_db(cfg, qdrantdb)

db_collection = qdrantdb.get_collection(cfg.db_collection)
vector_store = QdrantVectorStore(
    client=qdrantdb,
    collection_name=cfg.db_collection,
    aclient=aqdrantdb,
    prefer_grpc=True
)

2024-08-09 10:39:30.064 | INFO     | src.run.orchestrator:setup_db:22 - Deleting existing Qdrant collection review_rec_bot__009_hybrid_retriever__huggingface__Snowflake_snowflake_arctic_embed_m_v1_5...
2024-08-09 10:39:30.071 | INFO     | src.run.orchestrator:setup_db:25 - Deleting persisted storage context at data/009_hybrid_retriever/storage_context...
2024-08-09 10:39:30.072 | INFO     | src.run.orchestrator:setup_db:29 - Creating new Qdrant collection review_rec_bot__009_hybrid_retriever__huggingface__Snowflake_snowflake_arctic_embed_m_v1_5...
WARNI [llama_index.vector_stores.qdrant.base] Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


# Index Embeddings

In [22]:
import time
import pickle
from multiprocessing import set_start_method
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.node_parser import SemanticSplitterNodeParser
# from llama_index.core.node_parser import SentenceSplitter

In [23]:
chunker = SemanticSplitterNodeParser
chunker_cfg = {
    "buffer_size": 1,
    "breakpoint_percentile_threshold": 95,
    "embed_model": embed_model
}

# chunker = SentenceSplitter
# chunker_cfg = {
#     "chunk_size": 512,
#     "chunk_overlap": 10
# }

In [24]:
t0 = time.perf_counter()
# TODO: TO understand the differences between points_count and indexed_vector_counts.
# Here indexed_vector_counts = 0
db_collection_count = db_collection.points_count

if db_collection_count > 0 and ARGS.RECREATE_INDEX == False:
    logger.info(f"Loading index from existing DB...")
    with open(cfg.nodes_persist_fp, 'rb') as f:
        logger.info(f"Loading cached `nodes` at {cfg.nodes_persist_fp}...")
        nodes = pickle.load(f)

    logger.info(f"Loading Storage Context from {cfg.storage_context_persist_dp}...")
    docstore = SimpleDocumentStore.from_persist_dir(persist_dir=cfg.storage_context_persist_dp)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
else:
    logger.info(f"Creating new DB index...")
    from llama_index.core.extractors import TitleExtractor
    from llama_index.core.ingestion import IngestionPipeline, IngestionCache

    # create the pipeline with transformations
    pipeline = IngestionPipeline(
        transformations=[
            chunker(**chunker_cfg),
            embed_model,
        ],
        vector_store = vector_store
    )

    num_workers = None
    # TODO: I can get the preprocessing completed much quicker if I push the initial steps through a multiprocessing loop and then separately create the embeddings using the built-in batching already provided in the SentenceTransformer encode method.
    # Ref: https://github.com/run-llama/llama_index/issues/10104#issuecomment-1899401584
    # Currently setting num_workers leads to code simple hang
    # Ref: https://github.com/run-llama/llama_index/issues/10104
    # num_workers = os.cpu_count() - 1
    # os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    # set_start_method("spawn", force=True)  # it hangs without this line
    logger.info(f"Running Ingestion Pipeline with {num_workers=}...")
    nodes = await pipeline.arun(documents=documents, num_workers=num_workers, show_progress=True)

    # Define Docstore as an abstraction on top of nodes to easily manage (e.g. get node by id)
    docstore = SimpleDocumentStore()
    await docstore.async_add_documents(nodes)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    logger.info(f"Persisting Storage Context to {cfg.storage_context_persist_dp}...")
    storage_context.persist(cfg.storage_context_persist_dp)

index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)
t1 = time.perf_counter()

2024-08-09 10:39:30.604 | INFO     | __main__:<module>:19 - Creating new DB index...
2024-08-09 10:39:30.610 | INFO     | __main__:<module>:40 - Running Ingestion Pipeline with num_workers=None...


Parsing nodes:   0%|          | 0/5240 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/9 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/7 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/18 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/28 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1022/1022 [03:40<00:00,  4.64it/s]
2024-08-09 10:52:24.855 | INFO     | __main__:<module>:50 - Persisting Storage Context to data/009_hybrid_retriever/storage_context...


In [25]:
logger.info(f"Indexing {len(documents)} documents into VectorStoreIndex took {t1 - t0:,.0f}s")
logger.info(f"[COLLECT] {len(nodes)=}")

2024-08-09 10:52:29.525 | INFO     | __main__:<module>:1 - Indexing 5240 documents into VectorStoreIndex took 779s
2024-08-09 10:52:29.526 | INFO     | __main__:<module>:2 - [COLLECT] len(nodes)=10219


# Analyze Chunks

In [26]:
for i, node in enumerate(nodes[:5]):
    print(f"\n\n==========Node {i+1}============")
    print(node.metadata)
    print(node.get_text())



==========Node 1============
{'review_id': 'L0jv8c2FbpWSlfNC6bbUEA', 'user_id': 'bFPdtzu11Oi0f92EAcjqmg', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-10-13T22:50:47', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly':

# Construct Retriever

In [27]:
from llama_index.core.retrievers import VectorIndexRetriever

In [28]:
print(cfg.retrieval_cfg.model_dump_json(indent=2))

{
  "retrieval_top_k": 10,
  "retrieval_similarity_cutoff": null,
  "rerank_top_k": 5,
  "rerank_model_name": "BAAI/bge-reranker-large"
}


In [29]:
# configure retriever
vector_retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=cfg.retrieval_cfg.retrieval_top_k,
)

In [30]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=cfg.retrieval_cfg.retrieval_top_k,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

DEBUG [bm25s] Building index from IDs objects


In [31]:
from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    similarity_top_k=cfg.retrieval_cfg.retrieval_top_k,
    num_queries=1,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    # query_gen_prompt="...",  # we could override the query generation prompt here
)

## Test retrieval

In [32]:
from llama_index.core.response.notebook_utils import display_source_node

In [33]:
query = "where to find some cold brew coffee?"
retrieval_results = await retriever.aretrieve(query)
for node in retrieval_results:
    display_source_node(node, source_length=1000)
    print(node.get_content(metadata_mode=MetadataMode.EMBED))
    print("\n")

**Node ID:** 171b392d-6a81-493c-a9a0-d197abc2566f<br>**Similarity:** 0.016666666666666666<br>**Text:** Will definitely order from here again!<br>

review_stars: 5
biz_name: BAP
biz_address: 1224 South St
biz_city: Philadelphia
biz_state: PA
biz_categories: ['Korean', 'Restaurants']

Will definitely order from here again!




**Node ID:** 7b0b5fbb-fdf7-4006-8366-cf25eff27941<br>**Similarity:** 0.016666666666666666<br>**Text:** Pricey ( a ham and cheese croissant was $5.50)  as with anywhere in Santa Barbra but very delicious. Cute space ( a bit hard to find ). The green eggs and ham  breakfast biscuit is to die for.  ($10 I think) The Santa Barbra tri tip sandwich was excellent! Cold brew coffee was good.<br>

review_stars: 4
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries']

Pricey ( a ham and cheese croissant was $5.50)  as with anywhere in Santa Barbra but very delicious. Cute space ( a bit hard to find ). The green eggs and ham  breakfast biscuit is to die for.  ($10 I think) The Santa Barbra tri tip sandwich was excellent! Cold brew coffee was good.




**Node ID:** eb0d0583-a323-4295-86c4-256d98f63110<br>**Similarity:** 0.01639344262295082<br>**Text:** I would recommend this place!<br>

...

review_stars: 3
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries']

Barista was fast at least. Good latte and cold brew. I should have just ordered a pastry though.




**Node ID:** 21288120-31d2-4096-b381-db899c374707<br>**Similarity:** 0.015625<br>**Text:** I love this place. I come here probably twice a month.<br>

review_stars: 4
biz_name: BAP
biz_address: 1224 South St
biz_city: Philadelphia
biz_state: PA
biz_categories: ['Korean', 'Restaurants']

I love this place. I come here probably twice a month.




**Node ID:** 82d7016c-b1d5-4be6-a9bb-b31d7fbdeaf3<br>**Similarity:** 0.015625<br>**Text:** Service at the register was friendly. Cold brew was sooo good as were the sandwiches we got. Will most definitely be back.<br>

review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries']

Service at the register was friendly. Cold brew was sooo good as were the sandwiches we got. Will most definitely be back.




## Retrieval Evaluation

### Synthetic

In [34]:
nodes = list(docstore.docs.values())

In [35]:
from src.run.eval import RetrievalEvaluator

In [36]:
retrieval_evaluator = RetrievalEvaluator()
retrieval_evaluator.generate_synthetic_dataset(cfg, nodes)

2024-08-09 10:52:40.862 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:36 - Creating new retrieval eval dataset at data/009_hybrid_retriever/retrieval_synthetic_eval_dataset.json...
2024-08-09 10:52:40.863 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:40 - Sampling 20 nodes for retrieval evaluation...
2024-08-09 10:52:40.872 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:67 - Creating new synthetic retrieval eval dataset...
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:16<00:00,  1.21it/s]
2024-08-09 10:52:57.354 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:74 - Persisting synthetic retrieval eval dataset to data/009_hybrid_retriever/retrieval_synthetic_eval_dataset.json...


In [37]:
retrieval_evaluator.retrieval_eval_dataset.queries

{'20da5735-a1d0-460b-87a0-e9c6db65e249': "What are the parking options near LaFitte's in New Orleans?",
 'c9f780b4-be9d-400f-b0d1-fe5e116b14ff': 'Was the delivery received without damage?',
 'd6d2d05a-c3e1-4715-8c14-13d82282c94d': 'What grocery items are popular in Nashville?',
 '7cb5db42-beb9-4366-9818-00d48a1368e9': 'What are the worst reviews for local restaurants?',
 '42ef3b88-f635-4637-a8c1-99b9268fbc34': 'What ice cream flavors are popular during winter?',
 'd586598d-d1b7-4d88-8adc-2b34ff5db869': 'What are the best places to get ice cream in Nashville?',
 '1da24dff-ff52-41ce-b934-7c7f2c23c8b0': 'What are the sushi options available at Tuna Bar?',
 '0ce27b5f-7e39-48e7-9ef6-1512969a81b7': 'What are the best breakfast sandwiches in town?',
 '82c0c3ed-0a57-441c-948f-50c4ec3e7e98': 'What are the reasons customers are unhappy with Chase credit card services?',
 '0774f7c3-4176-447a-b6e5-7ec6d5972606': "What are the reviews for Tony's Alton restaurant?",
 '113874d9-7c19-4c41-92c5-ad80e22

In [38]:
retrieval_eval_results_df, retrieval_eval_results_full_df = await retrieval_evaluator.aevaluate(cfg, retriever)

In [39]:
# TODO: Why all zeros?
retrieval_eval_results_df

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top_10_retrieval_eval,0.55,0.262698,0.055,0.55,0.262698,0.073185


In [40]:
retrieval_eval_results_full_df

,query,expected_ids,retrieved_texts,hit_rate,mrr,precision,recall,ap,ndcg
0,What are the parking options near LaFitte's in...,[e398cffc-eb02-42e3-923a-604ae06c4889],[This is a little weird but works out fine.\n\...,1.0,0.500000,0.1,1.0,0.500000,0.138862
1,Was the delivery received without damage?,[03e033d4-2831-439e-8ea6-a4946a41916a],"[Received confirmation of my order at 5 pm, ch...",1.0,0.500000,0.1,1.0,0.500000,0.138862
2,What grocery items are popular in Nashville?,[7bb1304d-9638-4b37-89b3-b8c852ffeff2],"[It is a hit with my family, Dont expect anyth...",0.0,0.000000,0.0,0.0,0.000000,0.000000
3,What are the worst reviews for local restaurants?,[70f23ccc-9c0d-478c-a3fa-602529082d56],[I tried this restaurant because of the Yelp r...,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,What ice cream flavors are popular during winter?,[f89a64e7-2b86-4333-89fa-11a18dc7d897],"[It is a hit with my family, An ice cream shop...",0.0,0.000000,0.0,0.0,0.000000,0.000000
5,What are the best places to get ice cream in N...,[fd39867d-dddb-4ae8-abd2-9e0422dce67a],[My daughter and I came in and we have an amaz...,0.0,0.000000,0.0,0.0,0.000000,0.000000
6,What are the sushi options available at Tuna Bar?,[5c86dd25-9c62-4117-ad76-87823a901d02],"[Just had our first, and certainly not last, s...",0.0,0.000000,0.0,0.0,0.000000,0.000000
7,What are the best breakfast sandwiches in town?,[07042f7e-af55-4f78-9c1b-5259eaa0cd11],[There were long lines for food and check out ...,0.0,0.000000,0.0,0.0,0.000000,0.000000
8,What are the reasons customers are unhappy wit...,[2b20d945-b172-4a11-b82f-8c3238d42a16],[Every person I have spoken with has a differe...,1.0,0.111111,0.1,1.0,0.111111,0.066254
9,What are the reviews for Tony's Alton restaurant?,[955626ac-20b8-422e-9562-302188e37c11],[So....I want to be fair to Tony's but I also ...,1.0,0.142857,0.1,1.0,0.142857,0.073364


#### Error Analysis

In [41]:
retrieval_eval_irrelevance_df = (
    retrieval_eval_results_full_df
    .loc[lambda df: df['hit_rate'].lt(1)]
    .sort_values(['hit_rate', 'mrr', 'precision', 'recall', 'ap', 'ndcg'])
)
retrieval_eval_irrelevance_df

,query,expected_ids,retrieved_texts,hit_rate,mrr,precision,recall,ap,ndcg
2,What grocery items are popular in Nashville?,[7bb1304d-9638-4b37-89b3-b8c852ffeff2],"[It is a hit with my family, Dont expect anyth...",0.0,0.0,0.0,0.0,0.0,0.0
3,What are the worst reviews for local restaurants?,[70f23ccc-9c0d-478c-a3fa-602529082d56],[I tried this restaurant because of the Yelp r...,0.0,0.0,0.0,0.0,0.0,0.0
4,What ice cream flavors are popular during winter?,[f89a64e7-2b86-4333-89fa-11a18dc7d897],"[It is a hit with my family, An ice cream shop...",0.0,0.0,0.0,0.0,0.0,0.0
5,What are the best places to get ice cream in N...,[fd39867d-dddb-4ae8-abd2-9e0422dce67a],[My daughter and I came in and we have an amaz...,0.0,0.0,0.0,0.0,0.0,0.0
6,What are the sushi options available at Tuna Bar?,[5c86dd25-9c62-4117-ad76-87823a901d02],"[Just had our first, and certainly not last, s...",0.0,0.0,0.0,0.0,0.0,0.0
7,What are the best breakfast sandwiches in town?,[07042f7e-af55-4f78-9c1b-5259eaa0cd11],[There were long lines for food and check out ...,0.0,0.0,0.0,0.0,0.0,0.0
13,What are the best breakfast sandwiches at Cavi...,[c92d191f-fdbb-4442-98cb-a823ead8df82],[Everything about this location confused me at...,0.0,0.0,0.0,0.0,0.0,0.0
14,What are the best gift ideas for men?,[e0c02a01-66aa-4e58-afed-93f9e6b4ec16],"[This is one of the most tasteful, well-curate...",0.0,0.0,0.0,0.0,0.0,0.0
17,What activities can I do with my son that he w...,[e3102655-7da6-4b01-aa15-ed64966ebeda],"[Definitely try this place for yourself, He la...",0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
for i, row in retrieval_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n============Error #{i+1}=============\n\n")
    print(f"Query:\n{row.query}\n")
    expected_contexts = [json.loads(record.payload['_node_content'])['text'] for record in qdrantdb.retrieve(cfg.db_collection, ids=row.expected_ids)]
    expected_contexts = '\n\n'.join(expected_contexts)
    print(f"Expected Contexts:\n{expected_contexts}\n")
    contexts = ""
    for i, context in enumerate(row.retrieved_texts):
        contexts = f"{contexts}\n\nContext #{i+1}: {context}"
    print(f"Retrieved Contexts:\n{contexts}\n")



============Error #1=============


Query:
What grocery items are popular in Nashville?

Expected Contexts:
Very clean and modern. It'll be great for a lot of Nashville and tourist Instagrammers to get their pose on. 

As far as groceries, lots of curated items from both Nashville and beyond. They have McClures pickles... being from Michigan I got super pumped about those. They also carry Jenis Ice Cream pints which are down right dangerous. Great selection of coffee, sweets, cheeses, charcuterie and wine.

The only reason it's not a full 5 is the parking situation. All the lots in midtown are super expensive or you need change for meters (it's 2017 I don't carry anything that's monetary unless it's a card). If you do meters, it's $1.50 per hour in that area.  Paid lots are about $8 an hour.

Retrieved Contexts:


Context #1: It is a hit with my family

Context #2: Dont expect anything fancy. My boyfriend and I order take out there and the manager is always so friendly. I highly reco

# Response

In [43]:
from llama_index.core import get_response_synthesizer
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

from src.features.append_reference.custom_query_engine import ManualAppendReferenceQueryEngine

In [44]:
node_postprocessors = []

if cfg.retrieval_cfg.retrieval_similarity_cutoff is not None:
    node_postprocessors.append(SimilarityPostprocessor(similarity_cutoff=cfg.retrieval_cfg.retrieval_similarity_cutoff))

reranker = FlagEmbeddingReranker(model=cfg.retrieval_cfg.rerank_model_name, top_n=cfg.retrieval_cfg.rerank_top_k)
node_postprocessors.append(reranker)

response_synthesizer = get_response_synthesizer()
query_engine = ManualAppendReferenceQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=node_postprocessors,
)

## Test Query Engine

In [45]:
from llama_index.core.response.notebook_utils import (
    display_source_node,
    display_response,
)

In [46]:
question = "where to find good cold brew coffee?"
response = query_engine.query(question)
display_response(response, show_source=True, show_metadata=True, show_source_metadata=True)

**`Final Response:`** You can find good cold brew coffee at a coffee house that offers all the great styles and flavors of coffee, similar to any other coffee house you'll find.


Sources:
- [None](None)





#### Referenced Paragraphs
Article: **None**

> ...Pricey ( a ham and cheese croissant was $5.50)  as with anywhere in Santa Barbra but very delicious. Cute space ( a bit hard to find ). The green eggs and ham  breakfast biscuit is to die for.  ($10 I think) The Santa Barbra tri tip sandwich was excellent! Cold brew coffee was good. ...

---

**`Source Node 1/5`**

**Node ID:** 7b0b5fbb-fdf7-4006-8366-cf25eff27941<br>**Similarity:** 0.7123664021492004<br>**Text:** Pricey ( a ham and cheese croissant was $5.50)  as with anywhere in Santa Barbra but very delicio...<br>**Metadata:** {'review_id': 'Q3fPo_x6xKxafAzy1hFITg', 'user_id': 'ha2Lv7WnxvdYnRsqwL1apQ', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-07-17T00:20:41', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'True', 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'False', 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries'], 'biz_hours': {'Friday': '8:0-14:0', 'Monday': '0:0-0:0', 'Saturday': '8:0-14:0', 'Sunday': '8:0-14:0', 'Thursday': '8:0-14:0', 'Tuesday': '8:0-14:0', 'Wednesday': '8:0-14:0'}}<br>

---

...

**Node ID:** 36991e23-3f96-4bcd-ac95-70c84e20e5f0<br>**Similarity:** -2.0586793422698975<br>**Text:** If it's cold, come grab a coffee instead. They offer all the great styles and flavors of coffee a...<br>**Metadata:** {'review_id': 'GHAY9_Kpk-1hcq4Ehfmu1w', 'user_id': 'HpXm_E_MRQdN_Rv4cARviA', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 1, 'funny': 0, 'cool': 1, 'date': '2016-02-07T16:02:23', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': 'False', 'Caters': 'False', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': 'False', 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': None, 'HappyHour': None, 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'None', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': None, 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food'], 'biz_hours': {'Friday': '8:0-0:0', 'Monday': '8:0-23:0', 'Saturday': '8:0-0:30', 'Sunday': '8:0-23:0', 'Thursday': '8:0-23:0', 'Tuesday': '8:0-23:0', 'Wednesday': '8:0-23:0'}}<br>

---

**`Source Node 5/5`**

**Node ID:** 17d515f7-ee7b-4ad8-ba5e-7ade77e44922<br>**Similarity:** -2.2148277759552<br>**Text:** This review is for the delicious breakfast and coffee. The Green Breakfast Bowl and the Biscuit s...<br>**Metadata:** {'review_id': '4sGHeBRdn36yOX3KszhzCA', 'user_id': 'iGhGzpTg0c3J_tuTi9TOzQ', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 5, 'useful': 1, 'funny': 1, 'cool': 1, 'date': '2018-07-28T19:12:02', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'True', 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'False', 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries'], 'biz_hours': {'Friday': '8:0-14:0', 'Monday': '0:0-0:0', 'Saturday': '8:0-14:0', 'Sunday': '8:0-14:0', 'Thursday': '8:0-14:0', 'Tuesday': '8:0-14:0', 'Wednesday': '8:0-14:0'}}<br>

{'7b0b5fbb-fdf7-4006-8366-cf25eff27941': {'review_id': 'Q3fPo_x6xKxafAzy1hFITg',
  'user_id': 'ha2Lv7WnxvdYnRsqwL1apQ',
  'business_id': 'IDtLPgUrqorrpqSLdfMhZQ',
  'review_stars': 4,
  'useful': 0,
  'funny': 0,
  'cool': 0,
  'date': '2018-07-17T00:20:41',
  'biz_name': 'Helena Avenue Bakery',
  'biz_address': '131 Anacapa St, Ste C',
  'biz_city': 'Santa Barbara',
  'biz_state': 'CA',
  'biz_postal_code': '93101',
  'biz_latitude': 34.4144445,
  'biz_longitude': -119.6906718,
  'biz_stars': 4.0,
  'biz_review_count': 389,
  'biz_is_open': 1,
  'biz_attributes': {'AcceptsInsurance': None,
   'Alcohol': "u'none'",
   'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}",
   'BYOB': None,
   'BestNights': None,
   'BikeParking': 'True',
   'BusinessAcceptsBitcoin': None,
   'BusinessAcceptsCreditCards': 'True',
   'BusinessParking': "{'garage': False, 'street': True, 'v

## Response Evaluation

In [47]:
from src.run.eval import ResponseEvaluator

In [48]:
response_evaluator = ResponseEvaluator()

### Synthetic

In [49]:
response_eval_documents, response_synthetic_eval_dataset = response_evaluator.generate_synthetic_dataset(cfg, documents)

2024-08-09 10:53:04.188 | INFO     | src.run.eval.response:generate_synthetic_dataset:39 - Sampling 20 documents for response evaluation...
2024-08-09 10:53:04.194 | INFO     | src.run.eval.response:generate_synthetic_dataset:59 - Creating new response eval dataset at data/009_hybrid_retriever/response_synthetic_eval_dataset.json...
2024-08-09 10:53:04.194 | INFO     | src.run.eval.response:generate_synthetic_dataset:62 - Creating synthetic response eval dataset...


Parsing nodes:   0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.41s/it]
2024-08-09 10:53:38.186 | INFO     | src.run.eval.response:generate_synthetic_dataset:85 - Persisting synthetic response eval dataset at data/009_hybrid_retriever/response_synthetic_eval_dataset.json...


In [50]:
response_synthetic_eval_prediction_dataset = await response_synthetic_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.10s/it]


In [51]:
response_synthetic_mean_scores_df, response_synthetic_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_synthetic_eval_dataset,
    response_synthetic_eval_prediction_dataset,
    dataset_name="synthetic",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:56<00:00,  2.81s/it]


In [52]:
response_synthetic_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,3.25
mean_relevancy_score,0.80
mean_faithfulness_score,0.80


In [53]:
response_synthetic_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
0,What are some popular Japanese restaurants in ...,\n\nThere are several popular Japanese restaur...,0.0,3.0,0.0,[One of my favorite restaurants in Nashville. ...
1,What are the reviews for Romano's Macaroni Gri...,\n\nThere are mixed reviews for Romano's Macar...,1.0,3.0,1.0,[People behind us were complaining because of ...
2,What are the best wine bars in New Orleans?,"\nBased on the reviews, it seems that Copper V...",1.0,4.0,1.0,[On vacation to New Orleans and this wine bar ...
3,What are the best dishes to try at Copper Vine...,\n\nIt seems that the smothered rabbit and mus...,1.0,2.0,1.0,"[The food, the atmosphere, and the service was..."
4,What unique dessert options are available at M...,\n\nIt seems that Mike's Ice Cream in Nashvill...,1.0,3.0,1.0,"[After dinner last night, my friends and I nee..."
5,What are the best coffee shops in Edmonton?,\n\nIt seems that the quality of coffee is a c...,1.0,3.0,0.0,[Roast is new to the coffee house scene in Edm...
6,What are the best options for burgers in Wesle...,\n\nIt seems that a popular sports bar in Wesl...,0.0,3.0,0.0,[Best sports bar in the Wesley Chapel area! Al...
7,What are the happy hour deals at Bar One in Ph...,"\n\nDecent discounts on drinks, and a fairly v...",1.0,4.0,1.0,[They renovated the space well. The lighting &...
8,What are the best sports bars in Indianapolis?,"\n\nBased on the reviews, it seems that Twin P...",1.0,4.0,1.0,[One of the best sports bars we've been too an...
9,What are the best gastropubs in New Orleans?,\nIt's likely that a gastropub in a city known...,0.0,2.0,0.0,[New and cool spot in downtown New Orleans!\nG...


#### Error Analysis

In [54]:
response_synthetic_eval_dataset_dict = dict()
for example in response_synthetic_eval_dataset.examples:
    response_synthetic_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [55]:
response_synthetic_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
9,What are the best gastropubs in New Orleans?,\nIt's likely that a gastropub in a city known...,0.0,2.0,0.0,[New and cool spot in downtown New Orleans!\nG...
0,What are some popular Japanese restaurants in ...,\n\nThere are several popular Japanese restaur...,0.0,3.0,0.0,[One of my favorite restaurants in Nashville. ...
6,What are the best options for burgers in Wesle...,\n\nIt seems that a popular sports bar in Wesl...,0.0,3.0,0.0,[Best sports bar in the Wesley Chapel area! Al...
13,What are the best sports bars in Indianapolis?,"\n\nBased on the reviews, it seems that Twin P...",0.0,3.0,1.0,[One of the best sports bars we've been too an...
3,What are the best dishes to try at Copper Vine...,\n\nIt seems that the smothered rabbit and mus...,1.0,2.0,1.0,"[The food, the atmosphere, and the service was..."
16,What are the best coffee shops in Nashville?,\nIt seems that Mike's Ice Cream is a popular ...,1.0,2.0,1.0,[This place is epic - to the point that we vis...
5,What are the best coffee shops in Edmonton?,\n\nIt seems that the quality of coffee is a c...,1.0,3.0,0.0,[Roast is new to the coffee house scene in Edm...
1,What are the reviews for Romano's Macaroni Gri...,\n\nThere are mixed reviews for Romano's Macar...,1.0,3.0,1.0,[People behind us were complaining because of ...
4,What unique dessert options are available at M...,\n\nIt seems that Mike's Ice Cream in Nashvill...,1.0,3.0,1.0,"[After dinner last night, my friends and I nee..."
14,What are the best sports bars in Indianapolis ...,\n\nIf you're looking for a great spot to catc...,1.0,3.0,1.0,[One of the best sports bars we've been too an...


In [56]:
response_synthetic_eval_irrelevance_df = (
    response_synthetic_deep_eval_df
    .loc[lambda df: df['relevancy_score'].lt(1)]
    .sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])
)

for i, row in response_synthetic_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_synthetic_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")



==============Error #1===============


Query:
What are the best gastropubs in New Orleans?

Context:
New and cool spot in downtown New Orleans!
Great variety of wines and local beers. 

What a great find in New Orleans on a Monday evening. We slid in under the Happy Hour expiration and my husband had three local brews at a fab price. 

And not an understatement the food was amazing definitely worth checking out

Not to mention, the food was fantastic! I would recommend this restaurant to anybody in the New Orleans area.

If you're in New Orleans give this place a visit!! 

Answer:

It's likely that a gastropub in a city known for its rich culinary history and vibrant nightlife would offer a unique blend of traditional pub fare and high-quality ingredients. These establishments often have a cozy atmosphere, a wide selection of craft beers, and a menu that showcases creative twists on classic dishes.


Sources:
- [None](None)



----

Expected Answer:
One of the best gastropubs in New

### Manually Curated
Ref: https://docs.llamaindex.ai/en/stable/examples/llama_dataset/ragdataset_submission_template/#1c-creating-a-labelledragdataset-from-scratch-with-manually-constructed-examples

In [57]:
response_curated_eval_dataset = response_evaluator.generate_curated_dataset(cfg)

2024-08-09 10:54:52.275 | INFO     | src.run.eval.response:generate_curated_dataset:124 - Persisting curated response eval dataset at data/009_hybrid_retriever/response_curated_eval_dataset.json...


In [58]:
response_curated_eval_prediction_dataset = await response_curated_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.18s/it]


In [59]:
response_curated_mean_scores_df, response_curated_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_curated_eval_dataset,
    response_curated_eval_prediction_dataset,
    dataset_name="curated",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.52s/it]


In [60]:
response_curated_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,2.0
mean_relevancy_score,1.0
mean_faithfulness_score,1.0


In [61]:
response_curated_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
0,What are the recommended restaurants nearby Sa...,\nIt's always exciting to discover new restaur...,1.0,2.0,1.0,[Finally some good food in downtown Santa Barb...


#### Answers for target questions

In [62]:
from IPython.display import Markdown

In [63]:
from src.run.eval.manual_eval_dataset import MANUAL_EVAL_QA

In [64]:
for question, expected_answer in MANUAL_EVAL_QA:
    display(Markdown(f"\n\n### Question: {question}\n"))
    response = query_engine.query(question)
    display_response(response)



### Question: What are the recommended restaurants nearby Santa Barbara?


**`Final Response:`** It's always exciting to discover new culinary gems in a city. When looking for recommended restaurants in a new area, it's a good idea to ask locals or check online reviews for insider tips. You can also try searching for popular food blogs or social media platforms to see what's trending in the area. Additionally, many restaurants offer online menus and reservation systems, which can give you an idea of their offerings and availability.


Sources:
- [None](None)

### Error Analysis

In [65]:
response_curated_eval_dataset_dict = dict()
for example in response_curated_eval_dataset.examples:
    response_curated_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [66]:
response_curated_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
0,What are the recommended restaurants nearby Sa...,\nIt's always exciting to discover new restaur...,1.0,2.0,1.0,[Finally some good food in downtown Santa Barb...


In [67]:
response_curated_eval_irrelevance_df = (
    response_curated_deep_eval_df
    .loc[lambda df: df['relevancy_score'].lt(1)]
    .sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])
)

for i, row in response_curated_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_curated_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")

# Persist run metadata

In [68]:
from src.run.utils import parse_collect_log, flatten_dict

In [69]:
collect_info = parse_collect_log(collect_fp)

In [70]:
if ARGS.LOG_TO_MLFLOW:
    import mlflow

    logger.info("Logging [COLLECT] info to MLflow...")
    mlflow.log_params(collect_info)
    logger.info("Logging config to MLflow...")
    mlflow.log_params(flatten_dict(cfg.model_dump(), "cfg", sep='.'))
    logger.info(f"Logging Retrieval Synthetic Eval Results to MLflow...")
    retrieval_evaluator.log_to_mlflow(cfg)
    logger.info(f"Logging Response Eval Results to MLflow...")
    response_evaluator.log_to_mlflow(
        cfg,
        'synthetic',
        response_synthetic_mean_scores_df,
        response_synthetic_deep_eval_df
    )
    response_evaluator.log_to_mlflow(
        cfg,
        'curated',
        response_curated_mean_scores_df,
        response_curated_deep_eval_df
    )

2024-08-09 10:54:59.448 | INFO     | __main__:<module>:4 - Logging [COLLECT] info to MLflow...
2024-08-09 10:54:59.476 | INFO     | __main__:<module>:6 - Logging config to MLflow...
/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
2024-08-09 10:54:59.511 | INFO     | __main__:<module>:8 - Logging Retrieval Synthetic Eval Results to MLflow...
2024-08-09 10:54:59.694 | INFO     | __main

# Clean up

In [71]:
if ARGS.LOG_TO_MLFLOW:
    mlflow.end_run()

# Archive